In [9]:
# Import Packages
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast, GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import random
import time, math, copy
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchtext
from tokenizers import CharBPETokenizer
import tokenizers
import operator
from queue import PriorityQueue

from sklearn.model_selection import train_test_split
import pandas as pd

# specify GPU
device = torch.device("cuda")

# Import from external file
X_train = pd.read_csv("output/x_train.csv")
X_test = pd.read_csv("output/x_test.csv")

In [10]:
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 5000

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

In [11]:
# Set paths
train_path = 'output/x_train_nn.csv'
test_path = 'output/x_test_nn.csv'

# Save files
X_train[['question', 'prep_answer', 'cluster']].to_csv(train_path, index=False, header=False)
X_test[['question', 'prep_answer', 'cluster']].to_csv(test_path, index=False, header=False)

# Create pytorch variables
src = torchtext.data.Field(
    include_lengths=True,
    lower=True
    )
tgt = torchtext.data.Field(
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN],
    lower=True,
    is_target=True
    )
cluster = torchtext.data.Field(
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='csv',
        fields=[('src', src), ('tgt', tgt), ('cluster', cluster)],
        filter_pred=len_filter
    )

data_test = torchtext.data.TabularDataset(
        path=test_path, format='csv',
        fields=[('src', src), ('tgt', tgt), ('cluster', cluster)],
        filter_pred=len_filter
    )

In [12]:
src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
cluster.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

20 tokens from input vocab:
 ['<unk>', '<pad>', 'the', 'to', 'of', 'a', 'i', 'in', 'and', 'is', 'that', 'for', 'this', 'be', 'it', 'are', 'have', 'my', 'on', 'with']

20 tokens from output vocab:
 ['<unk>', '<pad>', 'the', 'to', 'of', 'and', 'a', 'is', 'in', 'that', 'for', 'you', 'not', 'are', 'be', 'it', 'or', 'with', 'have', 'as']

num training examples: 1034

example train data:
src:
 ['knowing', 'that', 'the', 'routes', 'of', 'transmissions', 'are', 'mainly', '(only?)', 'eye,', 'nose,', 'mouth:', 'https://www.who.int/news-room/commentaries/detail/modes-of-transmission-of-virus-causing-covid-19-implications-for-ipc-precaution-recommendations', 'would', 'it', 'be', 'possible', 'to', 'purposely', 'catch', 'it', 'in', 'a', 'specific', 'way', 'so', 'that', 'it', 'will', 'only', 'result', 'in', 'mild', 'symptoms?', 'maybe', 'even', 'infecting', 'other', 'mucus', 'areas', 'inside', 'the', 'body', '(by', "let's", 'say', 'injection)', 'where', 'it', 'cannot', 'spread', 'too', 'much.', 'it',

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super().__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
        self.rnn1 = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=self.hidden_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            batch_first=True
        )

    def forward(self, x):
        x = x.reshape((1, self.seq_len, self.n_features))
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        return hidden_n.reshape((self.n_features, self.embedding_dim))

    def init_hidden(self, tensor_size):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super().__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        self.rnn1 = nn.LSTM(
            input_size=input_dim,
            hidden_size=input_dim,
            num_layers=1,
            batch_first=True
            )
        self.rnn2 = nn.LSTM(
            input_size=input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
            )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)

    def forward(self, x):
        x = x.repeat(self.seq_len, self.n_features)
        x = x.reshape((self.n_features, self.seq_len, self.input_dim))
        x, (hidden_n, cell_n) = self.rnn1(x)
        x, (hidden_n, cell_n) = self.rnn2(x)
        x = x.reshape((self.seq_len, self.hidden_dim))
        return self.output_layer(x)

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

In [16]:
def train(input_tensor, target_tensor, cluster, batch_size, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):

    encoder.train()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # get an initial hidden state for the encoder
    encoder_hidden = encoder.init_hidden(input_length)

    # detach hidden states
    encoder_hidden = tuple([each.data for each in encoder_hidden])

    # zero accumulated gradients
    encoder.zero_grad()
    decoder.zero_grad()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # create empty tensor to fill with encoder outputs
    #encoder_outputs = torch.zeros(max_length, encoder.n_hidden, device=device)

    # create a variable for loss
    loss = 0

    # pass the inputs through the encoder
    #for ei in range(input_length):
    encoder_hidden = encoder(input_tensor, encoder_hidden)
        #encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=device)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # detach hidden states
    decoder_hidden = tuple([each.data for each in decoder_hidden])[0]

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

        topv, topi = decoder_output.topk(2)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))

        if use_teacher_forcing:
            decoder_input = target_tensor[di]

        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    nn.utils.clip_grad_norm_(encoder.parameters(), 1)
    nn.utils.clip_grad_norm_(decoder.parameters(), 1)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters(encoder, decoder, n_iters, batch_size=32, print_every=10000, learning_rate=0.04, teacher_forcing_ratio=0.2):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = AdamW(encoder.parameters(), lr=learning_rate)
    decoder_optim = AdamW(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=batch_size,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=device, repeat=False)


    criterion = nn.CrossEntropyLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1

            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            cluster = getattr(batch, 'cluster')

            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            #input_tensor = input_tensor[0]
            #target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, cluster, batch_size, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss


            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()

        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')

hidden_size = 256
encoder1 = EncoderRNN(len(input_vocab)).to(device)
decoder1 = DecoderRNN(128, hidden_size, len(output_vocab)).to(device)
trainIters(encoder1, decoder1, 10, print_every=10000)

d:\programdata\venv\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Running 10 epochs...


RuntimeError: Expected hidden size (2, 1, 256), got [4, 283, 256]

In [7]:
def trainIters(encoder, decoder, n_iters, batch_size=30, print_every=10000, learning_rate=0.04, teacher_forcing_ratio=0.2):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = AdamW(encoder.parameters(), lr=learning_rate)
    decoder_optim = AdamW(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=batch_size,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=device, repeat=False)


    criterion = nn.CrossEntropyLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1

            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            cluster = getattr(batch, 'cluster')

            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            #input_tensor = input_tensor[0]
            #target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, cluster, batch_size, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss


            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()

        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')

In [8]:
hidden_size = 256
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, hidden_size, len(output_vocab)).to(device)

d:\programdata\venv\lib\site-packages\torch\nn\modules\rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
trainIters(encoder1, decoder1, 10, print_every=10000)

Running 10 epochs...


RuntimeError: Expected hidden[0] size (4, 365, 256), got [4, 32, 256]